# Step 1. 데이터 다운로드

https://www.kaggle.com/paultimothymooney/poetry/data
    
    ```
    $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
    ```

# Step 2. 데이터 읽어오기

### glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하기

txt파일에서 데이터를 받아와 문장단위로 끝어서 raw_corpus에 저장


In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import glob
import os
import tensorflow as tf    # 대망의 텐서플로우!
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기(문장갯수):", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

### 문장의 갯수 추측하기
print('raw_corpus타입',type(raw_corpus))
print('첫번재문장[0]:',raw_corpus[0])
print('마지막문장[187087]:',raw_corpus[187087])

데이터 크기(문장갯수): 187088
Examples:
 ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you"]
raw_corpus타입 <class 'list'>
첫번재문장[0]: The first words that come out
마지막문장[187087]: Now I can't work like this, no, with you next to me 


# Step 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거해야 된다.   
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권한다.   



### 문장 출력하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    print('문장길이:',len(sentence)) #sentence len길이
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

문장길이: 29
The first words that come out
문장길이: 41
And I can see this song will be about you
문장길이: 46
I can't believe that I can breathe without you
문장길이: 32
But all I need to do is carry on
문장길이: 26
The next line I write down
문장길이: 47
And there's a tear that falls between the pages
문장길이: 45
I know that pain's supposed to heal in stages
문장길이: 77
But it depends which one I'm standing on I write lines down, then rip them up
문장길이: 86
Describing love can't be this tough I could set this song on fire, send it up in smoke
문장길이: 57
I could throw it in the river and watch it sink in slowly
문장길이: 48


### 한 문장을 정체화 하는 함수를 정의한다. (문자부호, 대소문자, 특수문자를 정제해준다.)

preprocess_sentence()를 정의한다.


Hi, my name is John. *("Hi," "my", …, "john." 으로 분리됨) - 문장부호

First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자

He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자

"1." 을 막기 위해 문장 부호 양쪽에 공백을 추가 할 거고요, "2." 를 막기 위해 모든 문자들을 소문자로 변환할 겁니다. "3."을 막기 위해 특수문자들은 모두 제거하도록 하죠!
    

출력 형식은 <start> 한문장 <end>


In [3]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


 소스문장(언어 모델의 입력 문장) :  \\<start\\> 나는 밥을 먹었다   
타켓문장(언어 모델의 출력 문장 ): 나는 밥을 먹었다 '\\<end\\>'    
    
    ```
    입력이 되는 문장 : 소스 문장(Source Sentence)   X_train
    정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(Target Sentence)   y_train 
     
    ```

#### 정제된 데이터 구축하기


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    #if len(sentence)>14 : continue
        
    corpus.append(preprocess_sentence(sentence))

print(len(corpus))

corpus[-10:]


175749


['<start> and i knew the truth , when it came , would be to that effect <end>',
 '<start> at least you re attracted to me which i did not expect <end>',
 '<start> didn t think you get my number down as such <end>',
 '<start> but i never hated myself for my age so much yeah and although my pride s , yeah , not easily disturbed , yeah <end>',
 '<start> you sent me flying when you kicked me to the curb <end>',
 '<start> so with your battered jeans yeah and your beastie tee <end>',
 '<start> now i can t work like this , no , with you next to me and although my pride s , yeah , not easily disturbed , yeah <end>',
 '<start> you sent me flying when you kicked me to the curb <end>',
 '<start> so with your battered jeans yeah and your beastie tee <end>',
 '<start> now i can t work like this , no , with you next to me <end>']

# 정제된 데이터 토큰화

tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 해준다.

이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 부른다.      
우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리된다고 한다.   

In [5]:
import tensorflow as tf    # 대망의 텐서플로우!

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    #len15_tensor = []
    #for txt in tensor:
        #if len(txt) < 16:
            #len15_tensor.append(txt)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    ########len15_tensor,maxlen=15, 추가########################################################
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print(tensor.shape)

[[   2    6  241 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2    5   32 ...    0    0    0]
 ...
 [   2    7 1020 ...    3    0    0]
 [   2   30   31 ...    0    0    0]
 [   5   32   15 ...   10   12    3]] <keras_preprocessing.text.Tokenizer object at 0x7efc0b1a70d0>
(175749, 15)


# 텐서로 표현된 정제된 데이터 출력

생성된 텐서 데이터를 3번째 행까지 출력

In [6]:
print(tensor[:5, :])

[[  2   6 241 426  17  66  56   3   0   0   0   0   0   0   0]
 [  2   8   5  32  64  42 334  88  27 113   7   3   0   0   0]
 [  2   5  32  15 217  17   5  32 795 258   7   3   0   0   0]
 [  2  36  24   5  90  10  47  26 836  18   3   0   0   0   0]
 [  2   6 327 431   5 732  60   3   0   0   0   0   0   0   0]]


# tokenizer에 구축된 단어 사전의 인덱스 출력

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
        
print(len(tokenizer.index_word))        

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
27592


# 패딩 추가하기



2 -> \<start\>    
3 -> \<\end\>    
0 -> \<pad\>     

In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(type(tensor))
print(tensor.shape)

print(src_input[0])
print(tgt_input[0])

<class 'numpy.ndarray'>
(175749, 15)
[  2   6 241 426  17  66  56   3   0   0   0   0   0   0]
[  6 241 426  17  66  56   3   0   0   0   0   0   0   0]


### tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
print(type(src_input))

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<class 'numpy.ndarray'>


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Step 4. 평가 데이터셋 분리

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, 
                                                          test_size=0.2, 
                                                          random_state=20)

# Step 5. 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) #히든레이어1012

#hidden_size = 2048
#model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) #히든레이어2048

In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.00790683e-04,  1.72643544e-04,  6.89882700e-06, ...,
         -1.51398315e-04, -1.85168989e-04,  7.19983582e-05],
        [ 1.34146423e-04,  3.36800586e-04, -9.82635902e-05, ...,
          3.65857632e-05, -4.65013843e-04,  4.74011758e-04],
        [ 1.27563675e-04,  3.24540684e-04, -3.29311297e-04, ...,
          1.39257027e-04, -5.36090520e-04,  6.24289911e-04],
        ...,
        [ 2.19937880e-03,  4.33785113e-04, -1.11543224e-03, ...,
         -4.87961166e-04, -1.81896123e-03, -4.05072089e-04],
        [ 2.52401060e-03,  1.25462946e-04, -1.15510426e-03, ...,
         -4.52192093e-04, -2.18216632e-03, -3.84058716e-04],
        [ 2.75579444e-03, -2.06585522e-04, -1.18993898e-03, ...,
         -4.00521152e-04, -2.52619525e-03, -3.37440782e-04]],

       [[ 1.00790683e-04,  1.72643544e-04,  6.89882700e-06, ...,
         -1.51398315e-04, -1.85168989e-04,  7.19983582e-05],
        [ 1.78171322e-04,  2.71307596e-04,  3

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)


In [15]:

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
686/686 [==============================] - 104s 151ms/step - loss: 3.6376 - val_loss: 3.2645
Epoch 2/10
686/686 [==============================] - 100s 146ms/step - loss: 3.1606 - val_loss: 3.0163
Epoch 3/10
686/686 [==============================] - 104s 152ms/step - loss: 2.9692 - val_loss: 2.8512
Epoch 4/10
686/686 [==============================] - 105s 153ms/step - loss: 2.8275 - val_loss: 2.7163
Epoch 5/10
686/686 [==============================] - 105s 154ms/step - loss: 2.7050 - val_loss: 2.5947
Epoch 6/10
686/686 [==============================] - 105s 153ms/step - loss: 2.5922 - val_loss: 2.4820
Epoch 7/10
686/686 [==============================] - 105s 154ms/step - loss: 2.4892 - val_loss: 2.3828
Epoch 8/10
686/686 [==============================] - 107s 156ms/step - loss: 2.3929 - val_loss: 2.2879
Epoch 9/10
686/686 [==============================] - 107s 155ms/step - loss: 2.3030 - val_loss: 2.1999
Epoch 10/10
686/686 [==============================] - 106s 155m

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

### 히든레이어 2048로 설정한 경우
(훈련과정 실수로 지워버렸는데 다시돌리기 시간걸려서 차이점만 확인해보자)

In [29]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , <end> '

In [30]:
generate_text(model, tokenizer, init_sentence="<start> big", max_len=20)

'<start> big tigger <end> '

In [31]:
generate_text(model, tokenizer, init_sentence="<start> your hands ")

'<start> your hands are mine <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> you are ")

'<start> you are my love , you are my heart <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> what the ")

'<start> what the fuck is this bitch inhalin ? <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> say ")

'<start> say it all <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> handsome ")

'<start> handsome <unk> <end> '

### 히든레이어 1024로 설정한 겨우

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> big", max_len=20)

'<start> big tigger <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> your hands ")

'<start> your hands are in the air <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> you are ")

'<start> you are the one that you know <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> what the ")

'<start> what the fuck is you ? <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> say ")

'<start> say it all , <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> handsome ")

'<start> handsome <unk> <end> '

#### 나중에 다시 볼 것들
```
- for src_sample, tgt_sample in dataset.take(1): break
- def __init__(self, vocab_size, embedding_size, hidden_size):'
- tf.data.Dataset.
- 슬라이싱
```

# 회고

- 256배치사이즈. 히든레이어 1024일때 random_state를 2000으로 설정했을때 val_loss가 점점 증가하서 9까지 올라갔다. 20인경우에는 잘 감소하였는데 왜 올라갔는지 잘 모르겠다.    
 random_state 는 재현가능(for reproducibility)하도록 난수의 초기값을 설정해주는 것이며, 아무 숫자나 넣어주면 됩니다. shuffle=True 가 디폴트 설정이므로 생략 가능하다라고 나온다.    
 https://rfriend.tistory.com/519 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]
 
 

- 이번 노드에서는 그럭저럭 따라 갈수 있었는는데 소스트레인데이터와 타겟트레인 데이터를 (124960,14)의 근사값으로 변경하는데 어려움을 많이 받았다. 팀원들 덕분에 어려움을 벗어날수 있었다. 
- 날이 갈수록 노드의 텍스트가 읽혀지지가 않는다. 컨디션이 안좋아서 그런지 모르겠지만 멘탈 관리좀 해야겠다.아무튼~.                             이번 노드는 다른노드보다 수월하다고 하였다.그런데 나는 어려웠다. 무엇인지는 알것같은데 바로 생각이 나지 않거나 적용이 되지 않았다. 그래서 이전 노드 왔다 갔다하면서 프로젝트를 수행하였다.(파이썬언어도 아직 어색하다. 시간 따로 잡아서 공부좀 해라...) 
허나 이번노드를 하면서 이전에 사용했던 부분들이 많이 겹쳐서 복습하는 듯한 느낌이 들어서 나름 괜찮았다.(그래도 어려웠다. 생소한것도 많다.)
- TextGenerator class를 지정할 때 super 개념이 있다고 한다. 같은 교육생 한분이 커뮤니티에 정보를 제공해주었다.항상 열심히 하시는 분들을 보면 감탄이 난다. 
 https://dojang.io/mod/page/view.php?id=2386
